In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import math


%matplotlib inline

plt.rc('axes', labelsize=20) 
plt.rcParams["figure.figsize"] = (10,10)
plt.rc('xtick', labelsize=13)
plt.rc('ytick', labelsize=13) 
mpl.rcParams['axes.titlesize'] = 24

In [2]:
from numpy import math

def fill_names(line, merged, not_merged):
    if not isinstance(line['candidate_name'], basestring):
        correct_name = not_merged.ix[not_merged.INE_ID == line.INE_ID]['candidate_name']
        if len(correct_name) == 1:
            merged.set_value(col='candidate_name', index=line.name, value=correct_name.iloc[0])
            
            len_before = len(not_merged)
            not_merged.drop(not_merged.index[not_merged.INE_ID == line.INE_ID], inplace=True)
            len_after = len(not_merged)
            assert len_before - len_after == 1
        else:
            if len(correct_name) > 1:
                print 'More than one citizens party in INE_ID ' + str(line.INE_ID)
            if len(correct_name) == 0:
                print 'Zero with citizens party in INE_ID ' + str(line.INE_ID)
                
def fill_ballot_order(line, merged, not_merged):
    if math.isnan(line['Order']):
        correct_ballot_order = not_merged.ix[not_merged.INE_ID == line.INE_ID]['Order']
        if len(correct_ballot_order) == 1:
            merged.set_value(col='Order', index=line.name, value=correct_ballot_order.iloc[0])
            
            len_before = len(not_merged)
            not_merged.drop(not_merged.index[not_merged.INE_ID == line.INE_ID], inplace=True)
            len_after = len(not_merged)
            assert len_before - len_after == 1
            
        else:
            if len(correct_ballot_order) > 1:
                print 'More than one citizens party in INE_ID ' + str(line.INE_ID)
            if len(correct_ballot_order) == 0:
                print 'Zero with citizens party in INE_ID ' + str(line.INE_ID)

# 2013

In [3]:
local_election_results_2013 = pd.read_csv('data/processed_data/local_elections_2013_results.csv', encoding='utf-8')
local_election_candidates_2013 = pd.read_csv('data/processed_data/local_elections_candidates_2013.csv', encoding='utf-8')
local_elections_order_2013 = pd.read_csv('data/processed_data/ballot-order-local-elections-2013.csv')

In [4]:
merged_results_candidates_2013 = pd.merge(
    local_election_results_2013, local_election_candidates_2013, 
    left_on=['INE_ID', 'Sigla', 'year'], right_on=['INE_ID', 'party', 'year'],
    how='outer')
merged_results_candidates_2013 = merged_results_candidates_2013.drop('party', 1)

len_everything_together = len(merged_results_candidates_2013)

not_merged_results_candidates_2013 = merged_results_candidates_2013[merged_results_candidates_2013.Concelho.isnull()]
merged_results_candidates_2013 = merged_results_candidates_2013[merged_results_candidates_2013.Concelho.notnull()]

assert len_everything_together == (len(not_merged_results_candidates_2013) + len(merged_results_candidates_2013))

In [5]:
# Some hacking to merge the rest of the entries
for i in range(len(merged_results_candidates_2013)):
    line = merged_results_candidates_2013.ix[i]
    fill_names(line, merged_results_candidates_2013, not_merged_results_candidates_2013)
    
assert len(not_merged_results_candidates_2013) == 0

In [6]:
merged_2013 = pd.merge(
    merged_results_candidates_2013, local_elections_order_2013, 
    left_on=['INE_ID', 'Sigla'], right_on=['INE_ID', 'Party'],
    how='outer'
)
merged_2013 = merged_2013.drop('Party', 1)

len_everything_together = len(merged_2013)

not_merged_2013 = merged_2013[merged_2013.Concelho.isnull()]
merged_2013 = merged_2013[merged_2013.Concelho.notnull()]

assert len_everything_together == (len(not_merged_2013) + len(merged_2013))

In [7]:
# Some hacking to merge the missing ballot order numbers
for i in range(len(merged_2013)):
    line = merged_2013.ix[i]
    fill_ballot_order(line, merged_2013, not_merged_2013)
    
# assert len(not_merged_2013) == 0

## There is a problem with the data. Here is the e-mail I've send to CNE (Nacional Comission of Elections) in hope to try to solve them

Estou a analisar dados das eleições autárquicas de 2013 e deparei-me com algo estranho. Estou-me a referir apenas a dados referentes a eleições para a Câmara Municipal.

No documento do sorteio das candidaturas (disponível aqui) estão presentes alguns partidos que depois não aparecem nos resultados das eleições (disponíveis aqui).

Por exemplo, segundo o documento do sorteio, para Ferreira do Zêzere, o partido B.E. supostamente estaria em primeiro lugar no boletim de voto. Mas nos resultados, este partido nem sequer aparece listado. 

Este problema acontece com os seguintes locais e partidos:

[TABLE]

A primeira coluna pode ser ignorada. INE_ID é o número de identificação dado ao local pelo Instituto Nacional de Estatística.

Porque é que isto acontece?

In [8]:
# Mark the Locations where the problem ocurrs 
merged_2013['problems_ballot_order'] = merged_2013.INE_ID.isin(not_merged_2013.INE_ID)

# 2009

In [9]:
local_election_results_2009 = pd.read_csv('data/processed_data/local_elections_2009_results.csv', encoding='utf-8')
local_election_candidates_2009 = pd.read_csv('data/processed_data/local_elections_candidates_2009.csv', encoding='utf-8')

In [10]:
merged_results_candidates_2009 = pd.merge(
    local_election_results_2009, local_election_candidates_2009, 
    left_on=['INE_ID', 'Sigla', 'year'], right_on=['INE_ID', 'party', 'year'],
    how='outer')
merged_results_candidates_2009 = merged_results_candidates_2009.drop('party', 1)

len_everything_together = len(merged_results_candidates_2009)

not_merged_results_candidates_2009 = merged_results_candidates_2009[merged_results_candidates_2009.Concelho.isnull()]
merged_results_candidates_2009 = merged_results_candidates_2009[merged_results_candidates_2009.Concelho.notnull()]

assert len_everything_together == (len(not_merged_results_candidates_2009) + len(merged_results_candidates_2009))

In [11]:
# Some hacking to merge the rest of the entries
for i in range(len(merged_results_candidates_2009)):
    line = merged_results_candidates_2009.ix[i]
    fill_names(line, merged_results_candidates_2009, not_merged_results_candidates_2009)
    
assert len(not_merged_results_candidates_2009) == 0

In [12]:
merged_2009 = merged_results_candidates_2009

# 2005

In [13]:
local_election_results_2005 = pd.read_csv('data/processed_data/local_elections_2005_results.csv', encoding='utf-8')

In [14]:
merged_2005 = local_election_results_2005

# Join data form all the years into just one data frame

In [260]:
elections_df = pd.concat([merged_2005, merged_2009, merged_2013])

In [261]:
import gender_guesser.detector as gender
gender_detector = gender.Detector()

def get_gender(name):
    
    if not isinstance(name, basestring):
        return name
    
    first_name = name.split(' ')[0]
    
    male_names = [u'João', u'Acílio', u'Litério', u'Hersílio', u'Agusto', u'Romão', u'Hélder', u'Dulcídio', u'Alano'
                  u'Gerónimo', u'Isaltino', u'Avantino', u'Atílio', u'Vitor', u'Beraldino', u'Estevão', u'Hernani',
                  u'Fábio', u'Juvenálio', u'Edegar', u'Bráulio', u'Vasques', u'Joviano', u'Alano', u'Gerónimo',
                  u'Honório', u'Albérico', u'Tomé', u'Gualter', u'Flamiano', u'Milcíades', u'Cílio', u'Parcidio',
                  u'Herlander', u'Lélio', u'Alírio', u'Patrique', u'Dinarte', u'Dírio', u'Jesus', u'Orlindo']
    female_names = [u'Sílvia', u'Ercília', u'Urãnia', u'Zuraida', u'Fermelinda', u'Brizelinda', u'Léli', u'Nair']
    
    if first_name in male_names:
        return u'male'
    
    if first_name in female_names:
        return u'female'
    
    return gender_detector.get_gender(first_name)


In [262]:
elections_df['gender'] = elections_df['candidate_name'].apply(get_gender)

In [263]:
elections_df.ix[elections_df.gender == u'unknown']

,Abstention (%),Brancos,Concelho,INE_ID,Inscritos,Mandatos,Nulos,Order,Sigla,Tipo,Votantes,Votos,Votos (%),candidate_name,problems_ballot_order,year,gender


In [264]:
elections_df.gender.unique()

array([nan, u'male', u'female'], dtype=object)

In [265]:
elections_df = elections_df.rename(columns={
    'Brancos': 'blank_votes',
    'Concelho': 'concelho',
    'Inscritos': 'enrolled',
    'Nulos': 'null_votes',
    'Mandatos': 'number_mandates',
    'Order': 'position_ballot',
    'Sigla': 'party_initials',
    'Tipo': 'party_type',
    'Votantes': 'voters',
    'Votos': 'votes',
    'Abstention (%)': 'abstention_%',
    'Votos (%)': 'votes_%',
})

In [266]:
new_columns_order = ['INE_ID','year', 'concelho', 'enrolled', 'voters', 'abstention_%', 'blank_votes', 'null_votes', 'party_initials', 'party_type', 'votes', 'votes_%', 'number_mandates', 'candidate_name', 'gender', 'position_ballot', 'problems_ballot_order']
assert len(new_columns_order) == len(list(elections_df))
elections_df = elections_df[new_columns_order]

In [267]:
# Some columns that should contain integers contain floats. Correct that. 
elections_df.year = elections_df.year.astype(int)
elections_df.enrolled = elections_df.enrolled.astype(int)
elections_df.voters = elections_df.voters.astype(int)
elections_df.votes = elections_df.votes.astype(int)
elections_df.number_mandates = elections_df.number_mandates.astype(int)
elections_df.blank_votes = elections_df.blank_votes.astype(int)
elections_df.null_votes = elections_df.null_votes.astype(int)
elections_df.null_votes = elections_df.null_votes.astype(int)
elections_df.index = elections_df.index.map(int)

In [268]:
# Complete votes_% column
elections_df['votes_%'] = (elections_df.votes / (elections_df.voters - elections_df.blank_votes - elections_df.null_votes)) * 100

In [269]:
# Correct error in one "party_initials" column value
index_wrong_party_initials = elections_df.index[(elections_df.year == 2013) & (elections_df.concelho == 'MARINHA GRANDE') & (elections_df.party_initials == '#NAME?')]
elections_df.set_value(col='party_initials', index=index_wrong_party_initials, value=u'+CMG')
print

In [270]:
# Uppercase all party initials
elections_df.party_initials = elections_df.party_initials.apply(lambda initials: initials.upper())

In [271]:
# Fill Party Type column

# Get known party_types
party_to_party_type_dict = {}
for i, party in elections_df[elections_df.party_type.notnull()][['party_initials', 'party_type']].drop_duplicates().iterrows():
    if party.party_initials in party_to_party_type_dict.keys():
        assert party_to_party_type_dict[party.party_initials] == party.party_type
    else:
        party_to_party_type_dict[party.party_initials] = party.party_type
        
# Manually complete the dict
party_to_party_type_dict[u'CDS-PP.PPD/PSD.PPM'] = u'Coligação'
party_to_party_type_dict[u'CDS-PP.PPM'] = u'Coligação'
party_to_party_type_dict[u'PND.PPM.'] = u'Coligação'
party_to_party_type_dict[u'PNR'] = u'Partido'
party_to_party_type_dict[u'I'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'II'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'III'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'IV'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'V'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'VII'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'XIX'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'X'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'XII'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'XIV'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'XV'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'XVI'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'XVII'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'XVIII'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'XX'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'PH'] = u'Partido'
party_to_party_type_dict[u'MMS'] = u'Partido'
party_to_party_type_dict[u'JPM'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'MEP'] = u'Partido'
party_to_party_type_dict[u'PSN'] = u'Partido'
party_to_party_type_dict[u'UPA'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'ISLV'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'VP'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'F.A.I'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'CP'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'A.C.F.'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'PPAS'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'F100%'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'OHS'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'MUDA'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'MAJUSP'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'AMCS'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'MON'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'CFC'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'M.D.I.'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'CFC'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'MCIA'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'MDVNS'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'ANTCM'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'TI'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'VLGC'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'MARCO'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'ICA'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'CIPA'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'MIC'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'NMMS'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'GIGA'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'TPL'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'M.I.C.'] = u'Grupos Cidadãos Eleitores'
party_to_party_type_dict[u'MIL'] = u'Grupos Cidadãos Eleitores'

# Get the null party_type values
elections_df.party_type = elections_df.party_initials.apply(
    lambda party_initials: party_to_party_type_dict[party_initials] if party_initials in party_to_party_type_dict.keys() else float('nan'), 1
)

print "This should be empty:"
elections_df[elections_df.party_type.isnull()]

This should be empty:


,INE_ID,year,concelho,enrolled,voters,abstention_%,blank_votes,null_votes,party_initials,party_type,votes,votes_%,number_mandates,candidate_name,gender,position_ballot,problems_ballot_order


In [272]:
# Translate party_type values to Engllish
party_type_translation = {
    u'Partido': u'Party',
    u'Coligação': u'Alliance',
    u'Grupos Cidadãos Eleitores': u'Independent'
}

elections_df.party_type = elections_df.party_type.apply(lambda party_type: party_type_translation[party_type])

In [273]:
elections_df

,INE_ID,year,concelho,enrolled,voters,abstention_%,blank_votes,null_votes,party_initials,party_type,votes,votes_%,number_mandates,candidate_name,gender,position_ballot,problems_ballot_order
0,101,2005,ÁGUEDA,41809,25435,39.163816,619,372,CDS-PP,Party,1432,5.858288,0,NaN,NaN,NaN,NaN
1,101,2005,ÁGUEDA,41809,25435,39.163816,619,372,PS,Party,11373,46.526755,4,NaN,NaN,NaN,NaN
2,101,2005,ÁGUEDA,41809,25435,39.163816,619,372,PCP-PEV,Alliance,835,3.415971,0,NaN,NaN,NaN,NaN
3,101,2005,ÁGUEDA,41809,25435,39.163816,619,372,B.E.,Party,551,2.254132,0,NaN,NaN,NaN,NaN
4,101,2005,ÁGUEDA,41809,25435,39.163816,619,372,PPD/PSD,Party,10253,41.944854,3,NaN,NaN,NaN,NaN
5,102,2005,ALBERGARIA-A-VELHA,20162,13197,34.545184,199,149,PPD/PSD,Party,7633,59.405401,4,NaN,NaN,NaN,NaN
6,102,2005,ALBERGARIA-A-VELHA,20162,13197,34.545184,199,149,PS,Party,1600,12.452331,1,NaN,NaN,NaN,NaN
7,102,2005,ALBERGARIA-A-VELHA,20162,13197,34.545184,199,149,PCP-PEV,Alliance,283,2.202506,0,NaN,NaN,NaN,NaN
8,102,2005,ALBERGARIA-A-VELHA,20162,13197,34.545184,199,149,CDS-PP,Party,3087,24.025216,2,NaN,NaN,NaN,NaN
9,102,2005,ALBERGARIA-A-VELHA,20162,13197,34.545184,199,149,B.E.,Party,246,1.914546,0,NaN,NaN,NaN,NaN


In [256]:
elections_df = elections_df.set_index('INE_ID')
elections_df.to_csv('data/processed_data/elections_camaras_municipais_portugal.csv', encoding='utf-8')